In [98]:
import numpy as np
import pandas as pd

from configparser import ConfigParser
config = ConfigParser()
# create your own config.ini in root of project folder to store project configurations
config.read('config.ini')


pathfile = config.get('main', 'dirty_csv')       

df = pd.read_csv(pathfile, nrows=10**6, encoding='ISO-8859-1')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)

df.head(200)



,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
0,53.0,32.0,-8.0,1642.0,1650,1545,65.0,NaN,0,NaN,4,10,4.0,1549.0,PIT,205.0,0,209,NaN,10,NaN,DCA,NaN,N443US,7.0,14.0,US,NaN,2002
1,164.0,155.0,-11.0,1754.0,1805,1610,175.0,NaN,0,NaN,4,2,0.0,1610.0,MCI,1072.0,0,109,NaN,12,NaN,MCO,NaN,N755,2.0,7.0,WN,NaN,1999
2,60.0,NaN,15.0,2005.0,1950,1850,60.0,NaN,0,NaN,5,10,15.0,1905.0,CLT,227.0,0,1276,NaN,12,NaN,ATL,NaN,NaN,NaN,NaN,DL,NaN,1993
3,51.0,NaN,-5.0,1818.0,1823,1728,55.0,NaN,0,NaN,4,28,-1.0,1727.0,BNA,200.0,0,961,NaN,9,NaN,MEM,NaN,NaN,NaN,NaN,AA,NaN,1989
4,45.0,29.0,2.0,1120.0,1118,1030,48.0,NaN,0,0.0,1,19,5.0,1035.0,CMH,116.0,0,5873,0.0,6,0.0,CVG,0.0,N785CA,3.0,13.0,OH,0.0,2006
5,49.0,37.0,2.0,1137.0,1135,1048,47.0,NaN,0,NaN,4,2,0.0,1048.0,CLT,156.0,0,353,NaN,1,NaN,MYR,NaN,N934VJ,6.0,6.0,US,NaN,1997
6,61.0,40.0,-3.0,1537.0,1540,1440,60.0,NaN,0,NaN,7,20,-4.0,1436.0,LAW,140.0,0,3281,NaN,7,NaN,DFW,NaN,N286AE,7.0,14.0,MQ,NaN,2008
7,150.0,126.0,-19.0,2015.0,2034,1745,169.0,NaN,0,NaN,4,15,0.0,1745.0,ATL,903.0,0,1521,NaN,10,NaN,PVD,NaN,N919DE,14.0,10.0,DL,NaN,1998
8,115.0,103.0,-5.0,735.0,740,640,120.0,NaN,0,NaN,2,16,0.0,640.0,SEA,689.0,0,1678,NaN,6,NaN,SLC,NaN,N346,2.0,10.0,WN,NaN,1998
9,NaN,NaN,NaN,NaN,730,605,85.0,B,1,0.0,7,10,NaN,NaN,SLC,391.0,0,4117,0.0,4,0.0,DEN,0.0,N705EV,0.0,0.0,EV,0.0,2005


In [21]:
del df

# ----------------------cleaning comments----------------------
- Total number of records > 120 million
- remove all records with more than certain number of NaN values?
- At least remove all records with NaNs for assumed FDS except for the ones where NaN has to be transformed to something (will see this for CarrierDelay below)?

- Transform all times to datetime to calculate with them?
- calculate datetime for DepTime by adding or subtracting DepDelay from CRSDepTime (Has Year, month, etc already in dataset) DateTime?
    - can be max 24 hours (1440 minutes) difference
- CRSDepTime when = 0 -> does not equal 00:00 but time is just not set!
    - when time = 1 -> 00:01
- CarrierDelay has 0.0 or NaN for when there is no delay? Turn NaN into 0?
    - Same goes for every other Delay column. Which delay columns do we want to keep?
- CancellationCode has letter for when Cancelled = 1 and NaN for when not cancelled
    - Change NaNs in CancellationCode to unique not-cancelled code?

- Remove records where TailNum = 0? Is normally several numbers and a letter. Or keep for soft functional dependency?
 

In [4]:
df.describe()

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,SecurityDelay,TaxiIn,TaxiOut,WeatherDelay,Year
count,978757.000000,681360.000000,978757.000000,978783.000000,1000000.000000,1000000.000000,999791.000000,1000000.000000,276290.000000,1000000.000000,1000000.000000,981117.000000,981117.000000,998310.000000,1000000.000000,1000000.000000,276290.000000,1000000.000000,276290.000000,276290.000000,696728.000000,696823.000000,276290.000000,1000000.000000
mean,120.218606,102.854060,7.051868,1492.796174,1490.388786,1333.926632,121.124092,0.018890,3.708991,3.941351,15.735314,8.158946,1347.962749,702.446892,0.002353,1364.394253,4.715715,6.556942,4.099834,0.024033,6.476692,15.218328,0.760534,1998.614886
std,68.704049,71.022941,30.809895,498.580434,493.228099,476.174529,68.105616,0.136137,19.515270,1.990578,8.793015,28.673504,476.983626,551.751271,0.048451,1403.895626,20.071980,3.447362,16.662616,0.954795,22.342821,11.073621,8.904026,6.224338
min,-525.000000,-1419.000000,-1410.000000,1.000000,0.000000,0.000000,-30.000000,0.000000,0.000000,1.000000,1.000000,-1410.000000,1.000000,11.000000,0.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1987.000000
25%,70.000000,54.000000,-7.000000,1115.000000,1115.000000,928.000000,71.000000,0.000000,0.000000,2.000000,8.000000,-2.000000,933.000000,307.000000,0.000000,450.000000,0.000000,4.000000,0.000000,0.000000,4.000000,10.000000,0.000000,1993.000000
50%,102.000000,84.000000,0.000000,1521.000000,1520.000000,1326.000000,102.000000,0.000000,0.000000,4.000000,16.000000,0.000000,1333.000000,545.000000,0.000000,945.000000,0.000000,7.000000,0.000000,0.000000,5.000000,13.000000,0.000000,1999.000000
75%,151.000000,133.000000,11.000000,1916.000000,1912.000000,1727.000000,152.000000,0.000000,0.000000,6.000000,23.000000,6.000000,1737.000000,937.000000,0.000000,1711.000000,0.000000,10.000000,0.000000,0.000000,7.000000,18.000000,0.000000,2004.000000
max,1812.000000,1570.000000,1435.000000,2717.000000,2400.000000,2359.000000,660.000000,1.000000,1126.000000,7.000000,31.000000,1439.000000,2635.000000,4983.000000,1.000000,9619.000000,613.000000,12.000000,875.000000,128.000000,1460.000000,1435.000000,958.000000,2008.000000


# check if special delays are only set when cancellation = 1

In [28]:
# Check if carrierdelay is set when cancellation = 0
df[(df['CarrierDelay'] > 0.0) & (df['CarrierDelay'].notnull())].head(20)

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
21,95.0,74.0,136.0,2343.0,2127,2035,112.0,NaN,0,136.0,7,30,153.0,2308.0,SGF,563.0,0,4811,0.0,7,0.0,ATL,0.0,N854AS,3.0,18.0,EV,0.0,2006
67,148.0,125.0,266.0,2156.0,1730,1402,148.0,NaN,0,259.0,4,1,266.0,1828.0,SJU,1045.0,0,1049,7.0,12,0.0,MIA,0.0,N623AA,3.0,20.0,AA,0.0,2005
106,269.0,233.0,48.0,2222.0,2134,1900,274.0,NaN,0,6.0,1,4,53.0,1953.0,SFO,1846.0,0,157,42.0,8,0.0,ORD,0.0,N452UA,7.0,29.0,UA,0.0,2003
141,71.0,53.0,16.0,1823.0,1807,1641,86.0,NaN,0,6.0,4,9,31.0,1712.0,CRP,354.0,0,3367,10.0,12,0.0,DFW,0.0,N630AE,4.0,14.0,MQ,0.0,2004
172,75.0,45.0,22.0,1140.0,1118,1006,72.0,NaN,0,22.0,5,20,19.0,1025.0,PSP,261.0,0,2741,0.0,10,0.0,PHX,0.0,N918FJ,5.0,25.0,YV,0.0,2006
197,178.0,150.0,38.0,2240.0,2202,1805,177.0,NaN,0,37.0,2,16,37.0,1842.0,FLL,1166.0,0,640,0.0,9,1.0,MDW,0.0,N303TZ,7.0,21.0,TZ,0.0,2003
204,72.0,58.0,104.0,1239.0,1055,940,75.0,NaN,0,104.0,6,21,107.0,1127.0,MHT,377.0,0,891,0.0,2,0.0,BWI,0.0,N343,3.0,11.0,WN,0.0,2004
227,109.0,97.0,41.0,2231.0,2150,2000,110.0,NaN,0,5.0,2,20,42.0,2042.0,SMF,647.0,0,2218,36.0,6,0.0,PHX,0.0,N642WN,4.0,8.0,WN,0.0,2006
318,83.0,64.0,18.0,2058.0,2040,1913,87.0,NaN,0,8.0,4,27,22.0,1935.0,DLH,397.0,0,4465,10.0,5,0.0,ORD,0.0,N847MQ,2.0,17.0,MQ,0.0,2004
322,165.0,140.0,31.0,1646.0,1615,1430,165.0,NaN,0,3.0,4,14,31.0,1501.0,DFW,984.0,0,1373,28.0,8,0.0,MCO,0.0,N449AA,16.0,9.0,AA,0.0,2008


# check strong FD (TailNum) -> (UniqueCarrier)

In [7]:
# ---------------- check strong FD (TailNum) -> (UniqueCarrier)---------------
# dependency is not a bi-implication
df_uniq_carr = df[['TailNum','UniqueCarrier']]
df_uniq_carr = df_uniq_carr.sort_values(by=['TailNum'])
# df_uniq_carr = df_uniq_carr.sort_values(by=['UniqueCarrier'])
df_uniq_carr.head(20)
# describe for for example unique value counts
df_uniq_carr.describe()

,TailNum,UniqueCarrier
count,696796,1000000
unique,13036,29
top,UNKNOW,DL
freq,4698,133601


In [ ]:
# check if dependency also works other way around. Seems not.
# Just pick one uniquecarrier value and filter NaNs on TailNum side
df_uniq_carr_filt = df_uniq_carr[(df_uniq_carr['UniqueCarrier'] == '9E') & (df_uniq_carr['TailNum'].notnull())]
# df_uniq_carr_filt.describe()
df_uniq_carr_filt.head(50)

In [13]:
df_uniq_carr[(df_uniq_carr['TailNum'].isnull())]

,TailNum,UniqueCarrier
2,NaN,DL
3,NaN,AA
10,NaN,CO
11,NaN,TW
14,NaN,AA
...,...,...
999987,NaN,US
999992,NaN,HP
999997,NaN,DL
999998,NaN,US


In [61]:
# check if TailNum values with weird encodings have same UniqueCarrier
df[(df['TailNum'].str.match('N81@@@', na=False))].sort_values(by=['TailNum']).head(200)

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
34,110.0,104.0,-2.0,2333.0,2335,2135,120.0,NaN,0,NaN,5,28,8.0,2143.0,HOU,687.0,0,551,NaN,12,NaN,STL,NaN,N81@@@,2.0,4.0,WN,NaN,2001
832571,78.0,73.0,-17.0,2328.0,2345,2110,95.0,NaN,0,NaN,7,18,0.0,2110.0,HOU,677.0,0,1583,NaN,2,NaN,ELP,NaN,N81@@@,3.0,2.0,WN,NaN,2001
831007,51.0,42.0,4.0,2019.0,2015,1920,55.0,NaN,0,NaN,4,26,8.0,1928.0,HRL,276.0,0,47,NaN,4,NaN,HOU,NaN,N81@@@,2.0,7.0,WN,NaN,2001
824725,60.0,45.0,25.0,1700.0,1635,1630,65.0,NaN,0,NaN,7,10,30.0,1700.0,MDW,284.0,0,505,NaN,6,NaN,CMH,NaN,N81@@@,5.0,10.0,WN,NaN,2001
799434,63.0,50.0,30.0,2225.0,2155,2050,65.0,NaN,0,NaN,4,12,32.0,2122.0,AMA,324.0,0,56,NaN,7,NaN,DAL,NaN,N81@@@,5.0,8.0,WN,NaN,2001
735273,78.0,68.0,-7.0,2108.0,2115,1950,85.0,NaN,0,NaN,2,6,0.0,1950.0,SMF,479.0,0,2083,NaN,3,NaN,PDX,NaN,N81@@@,3.0,7.0,WN,NaN,2001
679288,45.0,35.0,5.0,2135.0,2130,2040,50.0,NaN,0,NaN,5,20,10.0,2050.0,DAL,189.0,0,100,NaN,7,NaN,AUS,NaN,N81@@@,5.0,5.0,WN,NaN,2001
635241,53.0,43.0,-7.0,1143.0,1150,1050,60.0,NaN,0,NaN,6,24,0.0,1050.0,HRL,276.0,0,11,NaN,11,NaN,HOU,NaN,N81@@@,2.0,8.0,WN,NaN,2001
611427,108.0,99.0,8.0,858.0,850,810,100.0,NaN,0,NaN,2,30,0.0,810.0,ELP,562.0,0,120,NaN,1,NaN,DAL,NaN,N81@@@,2.0,7.0,WN,NaN,2001
494459,75.0,64.0,-10.0,1520.0,1530,1405,85.0,NaN,0,NaN,3,11,0.0,1405.0,HOU,453.0,0,1023,NaN,7,NaN,TUL,NaN,N81@@@,4.0,7.0,WN,NaN,2001


In [56]:
# not a strong dependency for this specific TailNum value
# there are probably more of these because of encodings
df[(df['TailNum'].str.match('N4èBãæ', na=False))].sort_values(by=['TailNum']).head(200)

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
170,151.0,115.0,10.0,1331.0,1321,1002,139.0,NaN,0,NaN,3,6,-2.0,1000.0,LGA,887.0,0,3140,NaN,2,NaN,STL,NaN,N4èBãæ,4.0,32.0,AA,NaN,2002
36354,103.0,74.0,40.0,1258.0,1218,1049,89.0,NaN,0,NaN,7,16,26.0,1115.0,STL,462.0,0,2882,NaN,12,NaN,OKC,NaN,N4èBãæ,3.0,26.0,TW,NaN,2001
310703,152.0,132.0,-7.0,749.0,756,621,155.0,NaN,0,NaN,7,10,-4.0,617.0,STL,813.0,0,2821,NaN,2,NaN,PHL,NaN,N4èBãæ,8.0,12.0,AA,NaN,2002
604644,188.0,151.0,-3.0,2136.0,2139,1937,182.0,NaN,0,NaN,1,4,-9.0,1928.0,STL,1068.0,0,3079,NaN,2,NaN,MIA,NaN,N4èBãæ,9.0,28.0,AA,NaN,2002
677361,77.0,42.0,15.0,1932.0,1917,1815,62.0,NaN,0,NaN,4,6,0.0,1815.0,STL,237.0,0,2746,NaN,12,NaN,MCI,NaN,N4èBãæ,15.0,20.0,TW,NaN,2001


In [60]:
df[(df['TailNum'].str.match('UNKNOW', na=False))]

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
36,NaN,NaN,NaN,NaN,1125,1005,80.0,NaN,1,NaN,6,30,NaN,NaN,ALB,318.0,0,360,NaN,12,NaN,DCA,NaN,UNKNOW,0.0,0.0,US,NaN,2000
177,NaN,NaN,NaN,NaN,805,655,70.0,NaN,1,NaN,3,14,NaN,NaN,PIT,259.0,0,527,NaN,6,NaN,RIC,NaN,UNKNOW,0.0,0.0,US,NaN,2000
257,NaN,NaN,NaN,NaN,1918,1815,63.0,NaN,1,NaN,2,15,NaN,NaN,MSP,232.0,0,1229,NaN,9,NaN,DSM,NaN,UNKNOW,0.0,0.0,NW,NaN,1998
390,NaN,NaN,NaN,NaN,1519,1400,79.0,NaN,1,NaN,2,18,NaN,NaN,RNO,415.0,0,2842,NaN,1,NaN,SNA,NaN,UNKNOW,0.0,0.0,AA,NaN,2000
640,NaN,NaN,NaN,NaN,835,653,102.0,NaN,1,NaN,1,31,NaN,NaN,PIT,485.0,0,133,NaN,1,NaN,MHT,NaN,UNKNOW,0.0,0.0,US,NaN,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999332,NaN,NaN,NaN,NaN,2054,1940,74.0,NaN,1,NaN,4,10,NaN,NaN,SFO,326.0,0,2228,NaN,2,NaN,BUR,NaN,UNKNOW,0.0,0.0,UA,NaN,2000
999498,NaN,NaN,NaN,NaN,1926,1740,166.0,NaN,1,NaN,1,21,NaN,NaN,STL,813.0,0,375,NaN,12,NaN,PHL,NaN,UNKNOW,0.0,0.0,TW,NaN,1998
999745,NaN,NaN,NaN,NaN,748,630,78.0,NaN,1,NaN,1,20,NaN,NaN,LAX,337.0,0,2001,NaN,3,NaN,SFO,NaN,UNKNOW,0.0,0.0,UA,NaN,2000
999898,NaN,NaN,NaN,NaN,2348,2320,28.0,NaN,1,NaN,7,7,NaN,NaN,HVN,45.0,0,1484,NaN,1,NaN,HPN,NaN,UNKNOW,0.0,0.0,UA,NaN,1996


In [57]:

df[(df['TailNum'].str.match('N986ä1', na=False))].sort_values(by=['TailNum']).head(200)

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
32,69.0,54.0,-12.0,1756.0,1808,1644,84.0,NaN,0,NaN,6,29,3.0,1647.0,MSP,334.0,0,765,NaN,9,NaN,ORD,NaN,N986ä1,5.0,10.0,UA,NaN,2001
207104,83.0,45.0,34.0,1556.0,1522,1324,58.0,NaN,0,NaN,5,26,9.0,1333.0,IND,177.0,0,1094,NaN,1,NaN,ORD,NaN,N986ä1,14.0,24.0,UA,NaN,2001
269329,69.0,55.0,-18.0,1209.0,1227,1103,84.0,NaN,0,NaN,3,24,-3.0,1100.0,IAD,326.0,0,277,NaN,1,NaN,BDL,NaN,N986ä1,5.0,9.0,UA,NaN,2001
289614,84.0,39.0,39.0,1929.0,1850,1644,66.0,NaN,0,NaN,5,20,21.0,1705.0,DAY,240.0,0,1754,NaN,7,NaN,ORD,NaN,N986ä1,5.0,40.0,UA,NaN,2001
404988,140.0,118.0,8.0,1952.0,1944,1835,129.0,NaN,0,NaN,6,24,-3.0,1832.0,ORD,589.0,0,1619,NaN,3,NaN,IAD,NaN,N986ä1,11.0,11.0,UA,NaN,2001
438780,56.0,43.0,-26.0,1337.0,1403,1345,78.0,NaN,0,NaN,6,20,-4.0,1341.0,ORD,264.0,0,1027,NaN,1,NaN,CVG,NaN,N986ä1,5.0,8.0,UA,NaN,2001
476523,35.0,22.0,-11.0,1243.0,1254,1210,44.0,NaN,0,NaN,5,23,-2.0,1208.0,CLT,83.0,0,540,NaN,2,NaN,GSO,NaN,N986ä1,5.0,8.0,UA,NaN,2001
553465,52.0,35.0,-6.0,1020.0,1026,930,56.0,NaN,0,NaN,5,7,-2.0,928.0,CID,196.0,0,1871,NaN,9,NaN,ORD,NaN,N986ä1,4.0,13.0,UA,NaN,2001
658792,75.0,60.0,71.0,1919.0,1808,1644,84.0,NaN,0,NaN,1,23,80.0,1804.0,MSP,334.0,0,765,NaN,4,NaN,ORD,NaN,N986ä1,2.0,13.0,UA,NaN,2001
727013,73.0,54.0,-12.0,1320.0,1332,1210,82.0,NaN,0,NaN,4,29,-3.0,1207.0,ORD,334.0,0,734,NaN,3,NaN,MSP,NaN,N986ä1,8.0,11.0,UA,NaN,2001


# check strong FD (Origin, Dest) -> (Distance)

In [112]:
df_origin_dest = df[['Origin', 'Dest', 'Distance']]
# df_origin_dest = df_origin_dest.sort_values(by=['Origin', 'Dest'])
df_origin_dest = df_origin_dest.sort_values(by=['Distance','Origin', 'Dest'])
# filter NaN
df_origin_dest[df_origin_dest['Distance'].notnull()].tail(500)
# compare amount of different groups of [origin, dest, distance] (full FD) with groups of [origin, dest] (left side FD)
# should be same number of groups for strong dependency
df_origin_dest.groupby(['Origin', 'Dest', 'Distance']).size().reset_index().rename(columns={0:'count'}).count()

Origin      6775
Dest        6775
Distance    6775
count       6775
dtype: int64

In [113]:
# counts for only left hand side FD
df_origin_dest.groupby(['Origin', 'Dest']).size().reset_index().rename(columns={0:'count'}).count()

Origin    6775
Dest      6775
count     6775
dtype: int64

# check Delta FD (DepTime) -> (CRSDepTime)

In [18]:
df_dep_time = df[['DepTime', 'CRSDepTime']]
df_dep_time = df_dep_time.sort_values(by=['DepTime'])
df_dep_time.head(20)

,DepTime,CRSDepTime
653823,1.0,2359
258351,1.0,2355
336885,1.0,2115
633307,1.0,1827
796290,1.0,1
374391,1.0,1
504480,1.0,2245
955063,1.0,1
527990,1.0,2355
21349,1.0,2340


In [27]:
# calculate datetime for DepTime by adding or subtracting DepDelay from CRSDepTime (Has Year, month, etc already in dataset) DateTime?
df[['CRSDepTime', 'DepTime', 'DepDelay']][df['DepDelay'] > 1000]
# 1440 is in minutes = 1 day -> no delays longer than one day
# When both CRSDepTime and DepTime are transformed to datetime with help of DepDelay CRSDepTime and DepTime can only differ by 1440 minutes max (1440/60 = 24 hours) 
# -> Delta = 24 hours
# Possibly also holds for arrival time
df[['CRSDepTime', 'DepTime', 'DepDelay']][df['DepDelay'] > 1440]

,CRSDepTime,DepTime,DepDelay
1147,1050,647.0,1197.0
12836,1231,620.0,1069.0
52286,1420,906.0,1126.0
128789,1651,1222.0,1171.0
139208,2010,1959.0,1429.0
148864,2008,1946.0,1418.0
159959,1058,545.0,1127.0
187232,1803,1759.0,1436.0
199664,1338,648.0,1030.0
282651,1514,828.0,1034.0


# check Soft FD (CRSDepTime, DepTime) -> (DepDelay)

In [14]:
df_crsdep_dep = df[['CRSDepTime', 'DepTime', 'DepDelay']]
df_crsdep_dep = df_crsdep_dep.sort_values(by=['CRSDepTime', 'DepTime'])
# df_crsdep_dep.head(500)
df_crsdep_dep.groupby(['CRSDepTime', 'DepTime', 'DepDelay']).size().reset_index().rename(columns={0:'count'}).head(100)
# df_crsdep_dep.count()
# CRSDepTime does not seem to be set on some occations (set to 0). Means that when CRSDepTime is set to 0 and DepTime are equal the DepDelay might differ.
df_crsdep_dep[df_crsdep_dep['CRSDepTime'] == 0].count()

# Check to see if there might be a strong FD when CRSDepTime is actually set
df_crsdep_dep[df_crsdep_dep['CRSDepTime'] != 0].head(50)
# Sometimes DepTime - CRSDepTime != DepDelay even when CRSDepTime is set -> Soft FD?
# This is because sometimes DepTime and CRSDepTime differ by a day and DepDelay is not calculated with DepTime - CRSDepTime!
# Look at Delta FD (DepTime) -> (CRSDepTime)

,CRSDepTime,DepTime,DepDelay
79802,1,1.0,0.0
148139,1,1.0,0.0
204629,1,1.0,0.0
236506,1,1.0,0.0
243306,1,1.0,0.0
247672,1,1.0,0.0
318031,1,1.0,0.0
336262,1,1.0,0.0
340122,1,1.0,0.0
363301,1,1.0,0.0


In [15]:
df_crsdep_dep[df_crsdep_dep['CRSDepTime'] != 0].groupby(['CRSDepTime', 'DepTime', 'DepDelay']).size().reset_index().rename(columns={0:'count'})

,CRSDepTime,DepTime,DepDelay,count
0,1,1.0,0.0,22
1,1,2.0,1.0,1
2,1,3.0,2.0,1
3,1,4.0,3.0,1
4,1,5.0,4.0,2
...,...,...,...,...
80865,2359,2356.0,-3.0,19
80866,2359,2357.0,-2.0,28
80867,2359,2358.0,-1.0,24
80868,2359,2359.0,0.0,51


In [16]:
df_crsdep_dep[df_crsdep_dep['CRSDepTime'] != 0].groupby(['CRSDepTime', 'DepTime']).size().reset_index().rename(columns={0:'count'})

,CRSDepTime,DepTime,count
0,1,1.0,22
1,1,2.0,1
2,1,3.0,1
3,1,4.0,1
4,1,5.0,2
...,...,...,...
79070,2359,2356.0,19
79071,2359,2357.0,28
79072,2359,2358.0,24
79073,2359,2359.0,51


# check Soft FD (ActualElapsedTime, TaxiIn, TaxiOut) -> (AirTime)

In [102]:
# ---------------- check Soft FD (ActualElapsedTime, TaxiIn, TaxiOut) -> (AirTime)---------------
df_air_time = df[['ActualElapsedTime', 'TaxiIn', 'TaxiOut', 'AirTime']]
df_air_time = df_air_time.sort_values(by=['ActualElapsedTime', 'TaxiIn', 'TaxiOut'])
df_air_time.head(50)

# Take a look at specific rows that have equal left hand side but AirTime is sometimes different.
df.iloc[[214815, 328863, 689521, 867367]]

# yet another few examples with same left hand side but sometimes different AirTime. Seems to be connection with CRSElapsedTime?
df.iloc[[119130, 615929, 673863]]


,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
119130,16.0,9.0,7.0,1056.0,1049,1029,20.0,NaN,0,NaN,1,27,11.0,1040.0,PSG,31.0,0,65,NaN,5,NaN,WRG,NaN,N747AS,3.0,4.0,AS,NaN,2002
615929,16.0,12.0,-7.0,1248.0,1255,1232,23.0,NaN,0,0.0,5,14,0.0,1229.0,VIS,36.0,0,6055,0.0,5,0.0,FAT,0.0,N292UX,3.0,4.0,OO,0.0,2004
673863,16.0,9.0,-2.0,1358.0,1400,1340,20.0,NaN,0,NaN,5,15,2.0,1342.0,ITH,32.0,0,1175,NaN,11,NaN,ELM,NaN,N886US,3.0,4.0,US,NaN,1996


# check Soft FD (AirTime, TaxiIn, TaxiOut) -> (ActualElapsedTime)

In [114]:
df_air_time_delay = df[['AirTime', 'TaxiIn', 'TaxiOut', 'ActualElapsedTime']]
df_air_time_delay = df_air_time_delay.sort_values(by=['AirTime', 'TaxiIn', 'TaxiOut'])
df_air_time_delay[df_air_time_delay['AirTime'].notnull() & df_air_time_delay['TaxiIn'].notnull() & df_air_time_delay['TaxiOut'].notnull()].head(500)





,AirTime,TaxiIn,TaxiOut,ActualElapsedTime
139136,-1419.0,1441.0,20.0,42.0
633330,-1417.0,1443.0,14.0,40.0
708954,-1414.0,1444.0,7.0,37.0
773258,-1413.0,1442.0,13.0,42.0
140584,-1413.0,1444.0,17.0,48.0
464072,-1410.0,1444.0,15.0,49.0
211788,-1410.0,1444.0,16.0,50.0
323928,-1410.0,1445.0,20.0,55.0
951785,-1409.0,1442.0,8.0,41.0
922405,-1409.0,1445.0,8.0,44.0


In [117]:
df_air_time_delay.groupby(['AirTime', 'TaxiIn', 'TaxiOut']).size().reset_index().rename(columns={0:'count'}).sort_values(by=['count'])

,AirTime,TaxiIn,TaxiOut,count
0,-1419.0,1441.0,20.0,1
78556,153.0,17.0,17.0,1
78555,153.0,17.0,16.0,1
78553,153.0,17.0,14.0,1
78551,153.0,17.0,10.0,1
...,...,...,...,...
10566,40.0,5.0,10.0,245
23799,60.0,5.0,10.0,321
13916,45.0,5.0,10.0,326
17268,50.0,5.0,10.0,340


In [116]:
df_air_time_delay.groupby(['AirTime', 'TaxiIn', 'TaxiOut', 'ActualElapsedTime']).size().reset_index().rename(columns={0:'count'})
# only +- 6.000 different groups between the two so soft dependency?

,AirTime,TaxiIn,TaxiOut,ActualElapsedTime,count
0,-1419.0,1441.0,20.0,42.0,1
1,-1417.0,1443.0,14.0,40.0,1
2,-1414.0,1444.0,7.0,37.0,1
3,-1413.0,1442.0,13.0,42.0,1
4,-1413.0,1444.0,17.0,48.0,1
...,...,...,...,...,...
136064,630.0,8.0,14.0,652.0,1
136065,637.0,6.0,26.0,669.0,1
136066,655.0,4.0,19.0,678.0,1
136067,1562.0,4.0,66.0,1812.0,1


In [135]:
# Take a look at specific rows that have equal left hand side but AirTime is sometimes different.
df.iloc[[494368, 508073, 645262, 968918]]
# again, seems to be soft dependency

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
494368,17.0,9.0,64.0,1837.0,1733,1714,19.0,NaN,0,NaN,3,17,66.0,1820.0,WRG,31.0,0,64,NaN,6,NaN,PSG,NaN,N744AS,3.0,5.0,AS,NaN,1998
508073,24.0,9.0,0.0,2143.0,2200,2136,24.0,NaN,0,NaN,5,17,0.0,2126.0,SGU,50.0,0,3622,NaN,1,NaN,CDC,NaN,N308SW,3.0,5.0,OO,NaN,2003
645262,17.0,9.0,6.0,1301.0,1255,1232,23.0,NaN,0,0.0,5,9,12.0,1244.0,VIS,36.0,0,6055,0.0,1,0.0,FAT,0.0,N292UX,3.0,5.0,OO,0.0,2004
968918,17.0,9.0,88.0,1200.0,1032,1010,22.0,NaN,0,0.0,6,9,93.0,1143.0,PSG,31.0,0,65,88.0,7,0.0,WRG,0.0,N783AS,3.0,5.0,AS,0.0,2005


# checking specific rows before preprocessing

In [15]:
# Causes many NaN values for assumed FDs. Also does not seem to create dependencies apart from Cancellation Code but even this one is not always set.
df[df['Cancelled'] == 1].head(200)

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
9,NaN,NaN,NaN,NaN,730,605,85.0,B,1,0.0,7,10,NaN,NaN,SLC,391.0,0,4117,0.0,4,0.0,DEN,0.0,N705EV,0.0,0.0,EV,0.0,2005
36,NaN,NaN,NaN,NaN,1125,1005,80.0,NaN,1,NaN,6,30,NaN,NaN,ALB,318.0,0,360,NaN,12,NaN,DCA,NaN,UNKNOW,0.0,0.0,US,NaN,2000
44,NaN,NaN,NaN,NaN,841,745,56.0,B,1,0.0,5,16,NaN,NaN,ASE,125.0,0,7047,0.0,2,0.0,DEN,0.0,N434YV,0.0,0.0,YV,0.0,2007
144,NaN,NaN,NaN,NaN,1130,1005,85.0,NaN,1,NaN,2,18,NaN,NaN,BWI,495.0,0,1162,NaN,2,NaN,SDF,NaN,NaN,0.0,0.0,WN,NaN,2003
177,NaN,NaN,NaN,NaN,805,655,70.0,NaN,1,NaN,3,14,NaN,NaN,PIT,259.0,0,527,NaN,6,NaN,RIC,NaN,UNKNOW,0.0,0.0,US,NaN,2000
257,NaN,NaN,NaN,NaN,1918,1815,63.0,NaN,1,NaN,2,15,NaN,NaN,MSP,232.0,0,1229,NaN,9,NaN,DSM,NaN,UNKNOW,0.0,0.0,NW,NaN,1998
301,NaN,NaN,NaN,NaN,949,701,108.0,NaN,1,NaN,2,29,NaN,NaN,MSP,680.0,0,986,NaN,12,NaN,DEN,NaN,NaN,NaN,NaN,UA,NaN,1987
390,NaN,NaN,NaN,NaN,1519,1400,79.0,NaN,1,NaN,2,18,NaN,NaN,RNO,415.0,0,2842,NaN,1,NaN,SNA,NaN,UNKNOW,0.0,0.0,AA,NaN,2000
560,NaN,NaN,NaN,NaN,1402,1235,87.0,NaN,1,NaN,5,4,NaN,NaN,DSM,490.0,0,768,NaN,3,NaN,MEM,NaN,NaN,NaN,NaN,NW,NaN,1988
639,NaN,NaN,NaN,NaN,1017,920,57.0,A,1,0.0,3,7,NaN,NaN,LAS,173.0,0,6016,0.0,7,0.0,PSP,0.0,0,0.0,0.0,OO,0.0,2004


In [18]:
df[(df['ArrDelay'].isnull()) & (df['Cancelled'] == 0)]

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
368,NaN,NaN,NaN,NaN,846,700,166.0,NaN,0,NaN,1,17,150.0,930.0,STL,872.0,1,451,NaN,5,NaN,EWR,NaN,NaN,NaN,NaN,CO,NaN,1993
598,NaN,NaN,NaN,NaN,2158,2040,78.0,NaN,0,0.0,5,13,115.0,2235.0,LAS,365.0,1,2892,0.0,10,0.0,TUS,0.0,N914FJ,0.0,14.0,YV,0.0,2006
986,NaN,NaN,NaN,NaN,1859,1725,154.0,NaN,0,NaN,4,10,69.0,1834.0,ORD,865.0,1,117,NaN,7,NaN,JAX,NaN,N415UA,NaN,14.0,UA,NaN,2008
1037,NaN,NaN,NaN,NaN,500,2100,300.0,NaN,0,0.0,1,17,26.0,2126.0,JFK,2465.0,1,358,0.0,4,0.0,BUR,0.0,N519JB,0.0,18.0,B6,0.0,2006
1107,NaN,NaN,NaN,NaN,2148,1905,163.0,NaN,0,NaN,1,10,19.0,1924.0,LGA,1076.0,1,194,NaN,7,NaN,FLL,NaN,NaN,NaN,NaN,TW,NaN,1989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997736,NaN,NaN,NaN,NaN,1135,1005,90.0,NaN,0,NaN,6,2,-3.0,1002.0,ISP,459.0,1,732,NaN,3,NaN,RDU,NaN,NaN,NaN,NaN,AA,NaN,1991
998506,NaN,NaN,NaN,NaN,1604,1430,94.0,NaN,0,NaN,1,20,10.0,1440.0,GFK,284.0,1,1203,NaN,7,NaN,MSP,NaN,NaN,NaN,NaN,NW,NaN,1992
999427,NaN,NaN,NaN,NaN,2216,2038,98.0,NaN,0,NaN,3,16,235.0,33.0,SFO,550.0,1,2322,NaN,2,NaN,PDX,NaN,N919UA,0.0,5.0,UA,NaN,2000
999619,NaN,NaN,NaN,NaN,1818,1236,222.0,NaN,0,NaN,6,22,53.0,1329.0,EWR,1725.0,1,1094,NaN,2,NaN,EGE,NaN,N17105,0.0,32.0,CO,NaN,2003


In [19]:
df[(df['DepDelay'].isnull()) & (df['Cancelled'] == 0)]

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year


In [20]:
# Causes many NaN values for assumed FDs. Also does not seem to create dependencies. Also only 2353 rows with diverted == 1.
df[(df['Diverted'] == 1)]

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
368,NaN,NaN,NaN,NaN,846,700,166.0,NaN,0,NaN,1,17,150.0,930.0,STL,872.0,1,451,NaN,5,NaN,EWR,NaN,NaN,NaN,NaN,CO,NaN,1993
598,NaN,NaN,NaN,NaN,2158,2040,78.0,NaN,0,0.0,5,13,115.0,2235.0,LAS,365.0,1,2892,0.0,10,0.0,TUS,0.0,N914FJ,0.0,14.0,YV,0.0,2006
986,NaN,NaN,NaN,NaN,1859,1725,154.0,NaN,0,NaN,4,10,69.0,1834.0,ORD,865.0,1,117,NaN,7,NaN,JAX,NaN,N415UA,NaN,14.0,UA,NaN,2008
1037,NaN,NaN,NaN,NaN,500,2100,300.0,NaN,0,0.0,1,17,26.0,2126.0,JFK,2465.0,1,358,0.0,4,0.0,BUR,0.0,N519JB,0.0,18.0,B6,0.0,2006
1107,NaN,NaN,NaN,NaN,2148,1905,163.0,NaN,0,NaN,1,10,19.0,1924.0,LGA,1076.0,1,194,NaN,7,NaN,FLL,NaN,NaN,NaN,NaN,TW,NaN,1989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997736,NaN,NaN,NaN,NaN,1135,1005,90.0,NaN,0,NaN,6,2,-3.0,1002.0,ISP,459.0,1,732,NaN,3,NaN,RDU,NaN,NaN,NaN,NaN,AA,NaN,1991
998506,NaN,NaN,NaN,NaN,1604,1430,94.0,NaN,0,NaN,1,20,10.0,1440.0,GFK,284.0,1,1203,NaN,7,NaN,MSP,NaN,NaN,NaN,NaN,NW,NaN,1992
999427,NaN,NaN,NaN,NaN,2216,2038,98.0,NaN,0,NaN,3,16,235.0,33.0,SFO,550.0,1,2322,NaN,2,NaN,PDX,NaN,N919UA,0.0,5.0,UA,NaN,2000
999619,NaN,NaN,NaN,NaN,1818,1236,222.0,NaN,0,NaN,6,22,53.0,1329.0,EWR,1725.0,1,1094,NaN,2,NaN,EGE,NaN,N17105,0.0,32.0,CO,NaN,2003


In [21]:
df[(df['TaxiIn'].isnull())]

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
2,60.0,NaN,15.0,2005.0,1950,1850,60.0,NaN,0,NaN,5,10,15.0,1905.0,CLT,227.0,0,1276,NaN,12,NaN,ATL,NaN,NaN,NaN,NaN,DL,NaN,1993
3,51.0,NaN,-5.0,1818.0,1823,1728,55.0,NaN,0,NaN,4,28,-1.0,1727.0,BNA,200.0,0,961,NaN,9,NaN,MEM,NaN,NaN,NaN,NaN,AA,NaN,1989
10,102.0,NaN,-4.0,1603.0,1607,1320,107.0,NaN,0,NaN,2,3,1.0,1321.0,DFW,641.0,0,280,NaN,12,NaN,DEN,NaN,NaN,NaN,NaN,CO,NaN,1991
11,87.0,NaN,-8.0,1547.0,1555,1322,93.0,NaN,0,NaN,3,13,-2.0,1320.0,CLE,487.0,0,142,NaN,11,NaN,STL,NaN,NaN,NaN,NaN,TW,NaN,1991
14,60.0,NaN,-5.0,2040.0,2045,1940,65.0,NaN,0,NaN,6,27,0.0,1940.0,DFW,247.0,0,559,NaN,10,NaN,SAT,NaN,NaN,NaN,NaN,AA,NaN,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999987,90.0,NaN,19.0,1702.0,1643,1400,103.0,NaN,0,NaN,4,2,32.0,1432.0,CLT,651.0,0,1838,NaN,11,NaN,MSY,NaN,NaN,NaN,NaN,US,NaN,1989
999992,50.0,NaN,-5.0,2020.0,2025,1930,55.0,NaN,0,NaN,6,14,0.0,1930.0,SAN,258.0,0,861,NaN,5,NaN,LAS,NaN,NaN,NaN,NaN,HP,NaN,1988
999997,256.0,NaN,10.0,1335.0,1325,1155,270.0,NaN,0,NaN,5,20,24.0,1219.0,LAX,1946.0,0,269,NaN,9,NaN,ATL,NaN,NaN,NaN,NaN,DL,NaN,1991
999998,65.0,NaN,4.0,1259.0,1255,1050,65.0,NaN,0,NaN,5,18,4.0,1054.0,PHX,325.0,0,80,NaN,1,NaN,ONT,NaN,NaN,NaN,NaN,US,NaN,1991


In [22]:
df[(df['TaxiIn'].isnull()) & (df['AirTime'].isnull())]

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
2,60.0,NaN,15.0,2005.0,1950,1850,60.0,NaN,0,NaN,5,10,15.0,1905.0,CLT,227.0,0,1276,NaN,12,NaN,ATL,NaN,NaN,NaN,NaN,DL,NaN,1993
3,51.0,NaN,-5.0,1818.0,1823,1728,55.0,NaN,0,NaN,4,28,-1.0,1727.0,BNA,200.0,0,961,NaN,9,NaN,MEM,NaN,NaN,NaN,NaN,AA,NaN,1989
10,102.0,NaN,-4.0,1603.0,1607,1320,107.0,NaN,0,NaN,2,3,1.0,1321.0,DFW,641.0,0,280,NaN,12,NaN,DEN,NaN,NaN,NaN,NaN,CO,NaN,1991
11,87.0,NaN,-8.0,1547.0,1555,1322,93.0,NaN,0,NaN,3,13,-2.0,1320.0,CLE,487.0,0,142,NaN,11,NaN,STL,NaN,NaN,NaN,NaN,TW,NaN,1991
14,60.0,NaN,-5.0,2040.0,2045,1940,65.0,NaN,0,NaN,6,27,0.0,1940.0,DFW,247.0,0,559,NaN,10,NaN,SAT,NaN,NaN,NaN,NaN,AA,NaN,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999987,90.0,NaN,19.0,1702.0,1643,1400,103.0,NaN,0,NaN,4,2,32.0,1432.0,CLT,651.0,0,1838,NaN,11,NaN,MSY,NaN,NaN,NaN,NaN,US,NaN,1989
999992,50.0,NaN,-5.0,2020.0,2025,1930,55.0,NaN,0,NaN,6,14,0.0,1930.0,SAN,258.0,0,861,NaN,5,NaN,LAS,NaN,NaN,NaN,NaN,HP,NaN,1988
999997,256.0,NaN,10.0,1335.0,1325,1155,270.0,NaN,0,NaN,5,20,24.0,1219.0,LAX,1946.0,0,269,NaN,9,NaN,ATL,NaN,NaN,NaN,NaN,DL,NaN,1991
999998,65.0,NaN,4.0,1259.0,1255,1050,65.0,NaN,0,NaN,5,18,4.0,1054.0,PHX,325.0,0,80,NaN,1,NaN,ONT,NaN,NaN,NaN,NaN,US,NaN,1991


In [23]:
df.groupby(['FlightNum']).count().reset_index().rename(columns={0:'count'})

,FlightNum,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
0,1,567,389,567,567,574,574,574,0,574,183,574,574,568,568,574,573,574,183,574,183,574,183,395,395,395,574,183,574
1,2,543,352,543,543,549,549,549,0,549,135,549,549,546,546,549,549,549,135,549,135,549,135,357,357,357,549,135,549
2,3,536,359,536,536,541,541,541,1,541,187,541,541,536,536,541,541,541,187,541,187,541,187,362,363,363,541,187,541
3,4,522,326,522,522,527,527,527,1,527,135,527,527,523,523,527,527,527,135,527,135,527,135,329,329,329,527,135,527
4,5,574,346,574,574,581,581,580,0,581,140,581,581,576,576,581,577,581,140,581,140,581,140,350,350,350,581,140,581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8000,9580,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8001,9582,3,3,3,3,3,3,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
8002,9583,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8003,9584,3,3,3,3,3,3,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [62]:
# df.sort_values(by=['FlightNum']).head(50)

# Flight number has many duplicate values that do not seem to correlate -> drop
df[df['FlightNum'] == 5843].sort_values(by=['Year']).head(20)

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
14948,77.0,65.0,-12.0,2137.0,2149,2030,79.0,NaN,0,0.0,4,5,-10.0,2020.0,SLC,402.0,0,5843,0.0,8,0.0,HLN,0.0,N510CA,9.0,3.0,OH,0.0,2004
43054,62.0,57.0,3.0,2152.0,2149,2030,79.0,NaN,0,0.0,2,24,20.0,2050.0,SLC,402.0,0,5843,0.0,8,0.0,HLN,0.0,N982CA,4.0,1.0,OH,0.0,2004
460251,85.0,65.0,0.0,2000.0,2000,1835,85.0,NaN,0,0.0,3,10,0.0,1835.0,HLN,402.0,0,5843,0.0,11,0.0,SLC,0.0,N814CA,4.0,16.0,OH,0.0,2004
955693,99.0,80.0,0.0,1839.0,1839,1700,99.0,NaN,0,0.0,4,15,0.0,1700.0,BOS,468.0,0,5843,0.0,12,0.0,ORF,0.0,N963CA,11.0,8.0,OH,0.0,2005
87378,136.0,118.0,-9.0,1204.0,1213,950,143.0,NaN,0,0.0,7,2,-2.0,948.0,RSW,879.0,0,5843,0.0,10,0.0,CVG,0.0,N642CA,7.0,11.0,OH,0.0,2005
364688,95.0,69.0,-9.0,1830.0,1839,1700,99.0,NaN,0,0.0,2,21,-5.0,1655.0,BOS,468.0,0,5843,0.0,3,0.0,ORF,0.0,N436CA,8.0,18.0,OH,0.0,2006
408867,NaN,NaN,NaN,NaN,1902,1729,153.0,C,1,0.0,3,29,NaN,NaN,ORD,773.0,0,5843,0.0,11,0.0,SAV,0.0,0,0.0,0.0,OO,0.0,2006
467879,70.0,48.0,-5.0,1820.0,1825,1710,75.0,NaN,0,0.0,3,27,0.0,1710.0,DSM,299.0,0,5843,0.0,9,0.0,ORD,0.0,N726SK,4.0,18.0,OO,0.0,2006
217478,91.0,70.0,-4.0,1601.0,1605,1429,96.0,NaN,0,0.0,5,8,1.0,1430.0,BZN,525.0,0,5843,0.0,6,0.0,DEN,0.0,N755SK,4.0,17.0,OO,0.0,2007
317924,89.0,54.0,10.0,949.0,939,825,74.0,NaN,0,0.0,7,9,-5.0,820.0,DSM,299.0,0,5843,0.0,9,0.0,ORD,0.0,N758SK,7.0,28.0,OO,0.0,2007


In [15]:
df[df['CRSDepTime'].isnull()]

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year


In [42]:
df.head(200)

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
0,53.0,32.0,-8.0,1642.0,1650,1545,65.0,NaN,0,NaN,4,10,4.0,1549.0,PIT,205.0,0,209,NaN,10,NaN,DCA,NaN,N443US,7.0,14.0,US,NaN,2002
1,164.0,155.0,-11.0,1754.0,1805,1610,175.0,NaN,0,NaN,4,2,0.0,1610.0,MCI,1072.0,0,109,NaN,12,NaN,MCO,NaN,N755,2.0,7.0,WN,NaN,1999
2,60.0,NaN,15.0,2005.0,1950,1850,60.0,NaN,0,NaN,5,10,15.0,1905.0,CLT,227.0,0,1276,NaN,12,NaN,ATL,NaN,NaN,NaN,NaN,DL,NaN,1993
3,51.0,NaN,-5.0,1818.0,1823,1728,55.0,NaN,0,NaN,4,28,-1.0,1727.0,BNA,200.0,0,961,NaN,9,NaN,MEM,NaN,NaN,NaN,NaN,AA,NaN,1989
4,45.0,29.0,2.0,1120.0,1118,1030,48.0,NaN,0,0.0,1,19,5.0,1035.0,CMH,116.0,0,5873,0.0,6,0.0,CVG,0.0,N785CA,3.0,13.0,OH,0.0,2006
5,49.0,37.0,2.0,1137.0,1135,1048,47.0,NaN,0,NaN,4,2,0.0,1048.0,CLT,156.0,0,353,NaN,1,NaN,MYR,NaN,N934VJ,6.0,6.0,US,NaN,1997
6,61.0,40.0,-3.0,1537.0,1540,1440,60.0,NaN,0,NaN,7,20,-4.0,1436.0,LAW,140.0,0,3281,NaN,7,NaN,DFW,NaN,N286AE,7.0,14.0,MQ,NaN,2008
7,150.0,126.0,-19.0,2015.0,2034,1745,169.0,NaN,0,NaN,4,15,0.0,1745.0,ATL,903.0,0,1521,NaN,10,NaN,PVD,NaN,N919DE,14.0,10.0,DL,NaN,1998
8,115.0,103.0,-5.0,735.0,740,640,120.0,NaN,0,NaN,2,16,0.0,640.0,SEA,689.0,0,1678,NaN,6,NaN,SLC,NaN,N346,2.0,10.0,WN,NaN,1998
9,NaN,NaN,NaN,NaN,730,605,85.0,B,1,0.0,7,10,NaN,NaN,SLC,391.0,0,4117,0.0,4,0.0,DEN,0.0,N705EV,0.0,0.0,EV,0.0,2005


# Actual preprocessing

In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan, when, count
import pyspark.sql.functions as F

from configparser import ConfigParser

config = ConfigParser()
# create your own config.ini in root of project folder to store project configurations
config.read('config.ini')

pathfile = config.get('main', 'dirty_csv')  

spark = SparkSession.builder \
    .config("spark.driver.memory", "15g") \
    .appName("SparkFlight").getOrCreate()



In [2]:
# data = spark.read.csv(pathfile, inferSchema='true', header='true', mode='PERMISSIVE', encoding='ISO-8859-1').limit(10**6).cache()
data = spark.read.csv(pathfile, inferSchema='true', header='true', mode='PERMISSIVE', encoding='ISO-8859-1')

### Analysis cells

In [67]:
data.show()

+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+
|               53|     32|      -8|   1642|      1650|      1545|            65|             

In [133]:
# check if read.csv(inferSchema='true') inferred the column types correctly
data.dtypes

[('ActualElapsedTime', 'string'),
 ('AirTime', 'string'),
 ('ArrDelay', 'string'),
 ('ArrTime', 'string'),
 ('CRSArrTime', 'int'),
 ('CRSDepTime', 'int'),
 ('CRSElapsedTime', 'string'),
 ('CancellationCode', 'string'),
 ('Cancelled', 'int'),
 ('CarrierDelay', 'string'),
 ('DayOfWeek', 'int'),
 ('DayofMonth', 'int'),
 ('DepDelay', 'string'),
 ('DepTime', 'string'),
 ('Dest', 'string'),
 ('Distance', 'string'),
 ('Diverted', 'int'),
 ('FlightNum', 'int'),
 ('LateAircraftDelay', 'string'),
 ('Month', 'int'),
 ('NASDelay', 'string'),
 ('Origin', 'string'),
 ('SecurityDelay', 'string'),
 ('TailNum', 'string'),
 ('TaxiIn', 'string'),
 ('TaxiOut', 'string'),
 ('UniqueCarrier', 'string'),
 ('WeatherDelay', 'string'),
 ('Year', 'int')]

In [37]:
data.count()

1000000

### {preprocess step} Filter out NA and Null values in all columns saved in var colnames

In [3]:
# filter out rows that have NA values in following columns that are assumed to be important for potential FDs:
colnames = ["ActualElapsedTime", "AirTime", "ArrDelay", "ArrTime", "CRSArrTime", "CRSDepTime", "CRSElapsedTime", "DayOfWeek", "DayofMonth", "DepDelay", "DepTime", "Dest", "Distance", "FlightNum", "Month", "Origin", "TailNum", "TaxiIn", "TaxiOut", "UniqueCarrier", "Year"]
# Specific delay columns like CarrierDelay and WeatherDelay with NA or Nul values need to have these values changed to 0 instead of filtered out.
# Tailnum still has some duplicated that do not have the same right UniqueCarrier
# also has some UNKNOW columns for TailNum. We keep these for soft dependency

# pyspark does not recognize csv "NA" strings as NaN values.
# This is reflected in the output of data.dtypes: columns that normally only have numeric values were not inferred as int type.
# Also remove these rows by filtering them out.
data_filt_nan = data.filter(data.ActualElapsedTime.isNotNull() & 
                            (data.ActualElapsedTime != "NA") &
                            data.AirTime.isNotNull() &
                            (data.AirTime != "NA") &
                            data.ArrDelay.isNotNull() &
                            (data.ArrDelay != "NA") &
                            data.ArrTime.isNotNull() &
                            (data.ArrTime != "NA") &              
                            data.CRSArrTime.isNotNull() & 
                            data.CRSDepTime.isNotNull() &
                            data.CRSElapsedTime.isNotNull() &
                            (data.CRSElapsedTime != "NA") &
                            data.DayOfWeek.isNotNull() &
                            data.DayofMonth.isNotNull() &
                            data.DepDelay.isNotNull() &
                            (data.DepDelay != "NA") &
                            data.DepTime.isNotNull() &
                            (data.DepTime != "NA") &
                            data.Dest.isNotNull() &
                            (data.Dest != "NA") &
                            data.Distance.isNotNull() &
                            (data.Distance != "NA") &
                            data.FlightNum.isNotNull() &
                            data.Month.isNotNull() &
                            data.Origin.isNotNull() &
                            (data.Origin != "NA") &
                            data.TailNum.isNotNull() &
                            (data.TailNum != "NA") &
                            data.TaxiIn.isNotNull() &
                            (data.TaxiIn != "NA") &
                            data.TaxiOut.isNotNull() &
                            (data.TaxiOut != "NA") &
                            data.UniqueCarrier.isNotNull() &
                            (data.UniqueCarrier != "NA") &
                            data.Year.isNotNull())

### Analysis cells

In [72]:
# this suggests the correct amount of rows are removed
data_filt_nan.count()

680914

In [70]:
# in the original dataframe there are still columns with NaN value that should be filtered out
data.select(*colnames).show()

+-----------------+-------+--------+-------+----------+----------+--------------+---------+----------+--------+-------+----+--------+---------+-----+------+-------+------+-------+-------------+----+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|FlightNum|Month|Origin|TailNum|TaxiIn|TaxiOut|UniqueCarrier|Year|
+-----------------+-------+--------+-------+----------+----------+--------------+---------+----------+--------+-------+----+--------+---------+-----+------+-------+------+-------+-------------+----+
|               53|     32|      -8|   1642|      1650|      1545|            65|        4|        10|       4|   1549| PIT|     205|      209|   10|   DCA| N443US|     7|     14|           US|2002|
|              164|    155|     -11|   1754|      1805|      1610|           175|        4|         2|       0|   1610| MCI|    1072|      109|   12|   MCO|   N755|     2|      7|           WN|1999|
|    

In [71]:
# NaN values in colnames should be filtered out
# TailNum still shows weird ascci signs sometimes.
data_filt_nan.select(*colnames).show(50)

+-----------------+-------+--------+-------+----------+----------+--------------+---------+----------+--------+-------+----+--------+---------+-----+------+-------+------+-------+-------------+----+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|FlightNum|Month|Origin|TailNum|TaxiIn|TaxiOut|UniqueCarrier|Year|
+-----------------+-------+--------+-------+----------+----------+--------------+---------+----------+--------+-------+----+--------+---------+-----+------+-------+------+-------+-------------+----+
|               53|     32|      -8|   1642|      1650|      1545|            65|        4|        10|       4|   1549| PIT|     205|      209|   10|   DCA| N443US|     7|     14|           US|2002|
|              164|    155|     -11|   1754|      1805|      1610|           175|        4|         2|       0|   1610| MCI|    1072|      109|   12|   MCO|   N755|     2|      7|           WN|1999|
|    

### {preprocess step} Take +- 15 mil rows now before any further transformations

In [4]:
data_limit = data_filt_nan.limit(15_000_000)

### Analysis cells

In [73]:
# data.printSchema()
# data.schema.names

['ActualElapsedTime',
 'AirTime',
 'ArrDelay',
 'ArrTime',
 'CRSArrTime',
 'CRSDepTime',
 'CRSElapsedTime',
 'CancellationCode',
 'Cancelled',
 'CarrierDelay',
 'DayOfWeek',
 'DayofMonth',
 'DepDelay',
 'DepTime',
 'Dest',
 'Distance',
 'Diverted',
 'FlightNum',
 'LateAircraftDelay',
 'Month',
 'NASDelay',
 'Origin',
 'SecurityDelay',
 'TailNum',
 'TaxiIn',
 'TaxiOut',
 'UniqueCarrier',
 'WeatherDelay',
 'Year']

In [53]:
# data_filt_nan

123395195

### {preprocess step} Transform certain existing columns that store minutes as string to timestamp type

In [5]:
# transform existing date columns to be able to use them for to_timestamp transformation
# this can then be used to calculate with timestamps for total time difference between two attributes (delta dependencies)
# example: transform 1 minute to hhm:mm -> 00:01
date_cols = ["Year", "Month", "DayofMonth", "CRSDepTime"]
data_timestamp = (data_limit.withColumn('Month', F.when(F.length(F.col('Month')) == 1, F.concat(F.lit('0'), F.col('Month'))).otherwise(F.col('Month')))
                    .withColumn('DayofMonth', F.when(F.length(F.col('DayofMonth')) == 1, F.concat(F.lit('0'), F.col('DayofMonth'))).otherwise(F.col('DayofMonth')))
                    .withColumn('CRSDepTime', F.when(F.length(F.col('CRSDepTime')) == 1, F.concat(F.lit('000'), F.col('CRSDepTime')))
                                        .when(F.length(F.col('CRSDepTime')) == 2, F.concat(F.lit('00'), F.col('CRSDepTime')))
                                        .when(F.length(F.col('CRSDepTime')) == 3, F.concat(F.lit('0'), F.col('CRSDepTime')))
                                        .otherwise(F.col('CRSDepTime')))
                    .withColumn('CRSDepTimeStamp', F.to_timestamp(F.concat(*date_cols), format='yyyyMMddHHmm'))
                 )

### Analysis cells

In [76]:
data_timestamp.show()

+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|    CRSDepTimeStamp|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+-------------------+
|               53|     32|      -

### {preprocess step} Transform DepTime with minutes only to full Timestamp with a date as new column using DepDelay

In [6]:
# Deptimestamp calculating by using CRSDepTimeStamp and DepDelay because actual DepTime and CRSDepTime can differ by at most one full day = 1440 mins
data_deptime = data_timestamp.withColumn("DepTimestamp", (col("CRSDepTimeStamp").cast("long") + (col("DepDelay").cast("long"))*60).cast("timestamp"))



### Analysis cells

In [84]:
data_deptime.show(500)

+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+-------------------+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|    CRSDepTimeStamp|       DepTimestamp|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+---------------

In [82]:
data_deptime.filter(data_deptime.DepDelay > 1300).show()

+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------------------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|            DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|    CRSDepTimeStamp|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------------------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+-------------------

### {preprocess step} Transform ArrTime with minutes only to full Timestamp with a date as new column by adding ActualElapsedTime to DepTimestamp 

In [7]:
data_arrtime = data_deptime.withColumn("ArrTimestamp", (col("DepTimestamp").cast("long") + (col("ActualElapsedTime").cast("long"))*60).cast("timestamp"))

### Analysis cells

In [87]:
data_arrtime.show(20)

+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+-------------------+-------------------+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|    CRSDepTimeStamp|       DepTimestamp|       ArrTimestamp|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------

### {preprocess step} Transform CRSArrTime with minutes only to full Timestamp with a date as new column by adding CRSElapsedTime to CRSDepTime 

In [8]:
# CRSArrTimestamp calculating by using CRSDepTimestamp and CRSElapsedTime
data_crs_arrtime = data_arrtime.withColumn("CRSArrTimestamp", (col("CRSDepTimestamp").cast("long") + (col("CRSElapsedTime").cast("long"))*60).cast("timestamp"))

### Analysis cells

In [89]:
data_crs_arrtime.show(20)

+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+-------------------+-------------------+-------------------+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|    CRSDepTimeStamp|       DepTimestamp|       ArrTimestamp|    CRSArrTimestamp|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+----

In [95]:
#  check if there are duplicates in DepTimestamp for delta dependency
data_crs_arrtime.groupBy("DepTimestamp").count().orderBy(col("count").desc()).show(500)

+-------------------+-----+
|       DepTimestamp|count|
+-------------------+-----+
|1999-06-21 23:57:00|    7|
|1999-03-02 23:59:00|    7|
|1995-11-08 00:00:00|    7|
|1999-08-31 23:59:00|    7|
|1999-03-01 23:57:00|    6|
|1999-03-15 00:00:00|    6|
|1999-03-07 23:58:00|    6|
|1997-12-16 23:59:00|    6|
|1995-11-12 23:59:00|    6|
|1999-03-09 23:59:00|    6|
|1995-12-02 23:59:00|    6|
|1996-01-06 00:00:00|    6|
|1995-03-08 07:00:00|    6|
|1999-10-06 23:58:00|    6|
|1999-03-02 23:57:00|    6|
|1999-03-01 23:59:00|    6|
|1995-12-02 00:00:00|    6|
|1999-11-16 23:59:00|    6|
|1997-11-30 23:59:00|    6|
|1999-03-01 00:00:00|    6|
|1998-01-26 00:00:00|    6|
|1999-11-17 00:00:00|    6|
|1999-11-05 23:59:00|    6|
|2004-04-08 09:30:00|    6|
|1999-11-05 23:58:00|    6|
|1995-11-16 00:00:00|    5|
|1999-02-28 23:57:00|    5|
|1999-10-01 23:58:00|    5|
|1998-01-24 00:00:00|    5|
|1999-02-28 23:56:00|    5|
|1996-12-07 00:00:00|    5|
|1995-08-11 00:00:00|    5|
|1998-01-29 23:58:00

In [96]:
#  check if there are duplicates in ArrTimestamp for delta dependency
data_crs_arrtime.groupBy("ArrTimestamp").count().orderBy(col("count").desc()).show(500)

+-------------------+-----+
|       ArrTimestamp|count|
+-------------------+-----+
|1999-09-27 17:30:00|    5|
|2006-10-04 16:30:00|    5|
|2004-09-18 19:43:00|    4|
|2001-08-08 11:24:00|    4|
|2007-02-06 09:20:00|    4|
|2004-07-03 16:54:00|    4|
|2004-02-05 18:51:00|    4|
|2005-01-05 15:48:00|    4|
|2001-05-08 15:49:00|    4|
|2006-07-19 17:40:00|    4|
|2004-04-29 13:06:00|    4|
|1997-07-27 11:34:00|    4|
|2001-09-02 09:46:00|    4|
|1999-10-31 15:08:00|    4|
|2006-03-26 15:50:00|    4|
|2000-12-05 23:20:00|    4|
|2006-03-10 18:04:00|    4|
|1998-06-25 21:32:00|    4|
|2004-04-22 11:45:00|    4|
|1997-06-14 19:22:00|    4|
|2008-09-01 14:23:00|    4|
|2001-01-18 20:12:00|    4|
|1997-11-17 13:02:00|    4|
|2008-03-02 13:56:00|    4|
|2004-01-14 12:50:00|    4|
|2006-06-04 16:22:00|    4|
|2007-09-03 18:21:00|    4|
|2005-08-28 10:35:00|    4|
|2003-07-15 18:39:00|    4|
|1997-11-26 19:29:00|    4|
|2007-03-27 12:01:00|    4|
|1996-11-17 21:42:00|    4|
|1997-10-10 18:10:00

### {preprocess step} Filter out rows that have have cancelled == 1 or Diverted == 1 as they leave out very valuable info
### This is done before removing these columns to filter out low information rows first

In [9]:
# Rows with cancelled == 1 or Diverted == 1 were probably already filtered because of filtered out NaN values for the important time columns
# Just to be sure
data_filt_div_canc = data_crs_arrtime.filter((data_crs_arrtime.Cancelled == 0) | (data_crs_arrtime.Diverted == 0))

### Analysis cells

In [123]:
# check if there are no rows left with cancelled == 1
data_filt_div_canc.orderBy(col("Cancelled").desc()).show(20)

+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+-------------------+-------------------+-------------------+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|    CRSDepTimeStamp|       DepTimestamp|       ArrTimestamp|    CRSArrTimestamp|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+----

### {preprocess step} Drop columns that have already been transformed to other columns or columns that seem less useful
- Original dataset has 28 columns which we want to try to reduce due to computational complexity
- The date columns like Year and Month have already been transformed to CRSDepTimeStamp
- Flightnumber only has a few thousand unique numbers over a million records and seems to have no correlation with others
- CancellationCode, Cancelled and Diverted seem to be very messy and occur only very rarely
    - Cancelled and Diverted only have two possible values: 1 and 0 and 1 only occurs very rarely
    - CancellationCode is also not always set when a flight is cancelled
- After this step number of cols is reduced from 28 -> 21

In [10]:
columns_to_drop = ["CancellationCode", "Cancelled", "Diverted", "Year", "Month", "DayOfWeek", "DayofMonth", "ArrTime", "CRSArrTime", "CRSDepTime", "DepTime", "FlightNum"]
# drop as many (mostly) useless columns as we can
data_drop_cols = data_filt_div_canc.drop(*columns_to_drop)

### Analysis cells

In [135]:
# data_drop_cols.show(20)
len(data_drop_cols.columns)

21

In [136]:
data_drop_cols.columns

['ActualElapsedTime',
 'AirTime',
 'ArrDelay',
 'CRSElapsedTime',
 'CarrierDelay',
 'DepDelay',
 'Dest',
 'Distance',
 'LateAircraftDelay',
 'NASDelay',
 'Origin',
 'SecurityDelay',
 'TailNum',
 'TaxiIn',
 'TaxiOut',
 'UniqueCarrier',
 'WeatherDelay',
 'CRSDepTimeStamp',
 'DepTimestamp',
 'ArrTimestamp',
 'CRSArrTimestamp']

### {preprocess step} Combine several specific delay columns to one column to reduce amount of columns
### Also cast columns that store minutes to int (turned out to be bigint)


In [11]:
# Combine LateAircraftDelay and CarrierDelay into one column.
# Combine NASDelay and WeatherDelay into one column
# Cast columns with string type to bigint (will get transformed later)
def transform_and_cast_cols(x):
    ActualElapsedTime= int(x.ActualElapsedTime)
    AirTime= int(x.AirTime)
    ArrDelay= int(x.ArrDelay)
    CRSElapsedTime= int(x.CRSElapsedTime)
    DepDelay= int(x.DepDelay)
    Dest= x.Dest
    Distance= int(x.Distance)
    LateAircraftDelay = x.LateAircraftDelay
    CarrierDelay = x.CarrierDelay
    
    if LateAircraftDelay == "NA":
        LateAircraftDelay = 0
    if CarrierDelay == "NA":
        CarrierDelay = 0 
    
    LateAircraftAndCarrierDelay = int(LateAircraftDelay) + int(CarrierDelay)
    
    NASDelay = x.NASDelay
    WeatherDelay = x.WeatherDelay
    if NASDelay == "NA":
        NASDelay = 0
    if WeatherDelay == "NA":
        WeatherDelay = 0 
    
    NASAndWeatherDelay= int(NASDelay) + int(WeatherDelay)
    Origin=x.Origin
    SecurityDelay = x.SecurityDelay
    if SecurityDelay == "NA":
        SecurityDelay = 0
        
    SecurityDelay= int(SecurityDelay)
    TailNum=x.TailNum
    TaxiIn=int(x.TaxiIn)
    TaxiOut=int(x.TaxiOut)
    UniqueCarrier=x.UniqueCarrier
    CRSDepTimeStamp=x.CRSDepTimeStamp
    DepTimestamp=x.DepTimestamp
    ArrTimestamp=x.ArrTimestamp
    CRSArrTimestamp=x.CRSArrTimestamp
    return (ActualElapsedTime,
            AirTime,
            ArrDelay,
            CRSElapsedTime,
            DepDelay,
            Dest,
            Distance,
            LateAircraftAndCarrierDelay,
            NASAndWeatherDelay,
            Origin,
            SecurityDelay,
            TailNum,
            TaxiIn,
            TaxiOut,
            UniqueCarrier,
            CRSDepTimeStamp,
            DepTimestamp,
            ArrTimestamp,
            CRSArrTimestamp)


In [12]:
# first transform dataframe to rdd as only rdd has map transform function
rdd_trans_n_cast_cols = data_drop_cols.rdd.map(lambda x: transform_and_cast_cols(x))  

In [13]:
# transform rdd back to dataframe
# have to specify columns again because rdd has to tabular structure
data_trans_n_cast_cols = rdd_trans_n_cast_cols.toDF(['ActualElapsedTime',
                                 'AirTime',
                                 'ArrDelay',
                                 'CRSElapsedTime',
                                 'DepDelay',
                                 'Dest',
                                 'Distance',
                                 'LateAircraftAndCarrierDelay',
                                 'NASAndWeatherDelay',
                                 'Origin',
                                 'SecurityDelay',
                                 'TailNum',
                                 'TaxiIn',
                                 'TaxiOut',
                                 'UniqueCarrier',
                                 'CRSDepTimeStamp',
                                 'DepTimestamp',
                                 'ArrTimestamp',
                                 'CRSArrTimestamp'])

### Analysis cells

In [150]:
data_trans_n_cast_cols.show(20)

+-----------------+-------+--------+--------------+--------+----+--------+---------------------------+------------------+------+-------------+-------+------+-------+-------------+-------------------+-------------------+-------------------+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|CRSElapsedTime|DepDelay|Dest|Distance|LateAircraftAndCarrierDelay|NASAndWeatherDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|    CRSDepTimeStamp|       DepTimestamp|       ArrTimestamp|    CRSArrTimestamp|
+-----------------+-------+--------+--------------+--------+----+--------+---------------------------+------------------+------+-------------+-------+------+-------+-------------+-------------------+-------------------+-------------------+-------------------+
|               53|     32|      -8|            65|       4| PIT|     205|                          0|                 0|   DCA|            0| N443US|     7|     14|           US|2002-10-10 15:45:00|2002-10-10 15:49:00|2

In [152]:
data_trans_n_cast_cols.dtypes

[('ActualElapsedTime', 'bigint'),
 ('AirTime', 'bigint'),
 ('ArrDelay', 'bigint'),
 ('CRSElapsedTime', 'bigint'),
 ('DepDelay', 'bigint'),
 ('Dest', 'string'),
 ('Distance', 'bigint'),
 ('LateAircraftAndCarrierDelay', 'bigint'),
 ('NASAndWeatherDelay', 'bigint'),
 ('Origin', 'string'),
 ('SecurityDelay', 'bigint'),
 ('TailNum', 'string'),
 ('TaxiIn', 'bigint'),
 ('TaxiOut', 'bigint'),
 ('UniqueCarrier', 'string'),
 ('CRSDepTimeStamp', 'timestamp'),
 ('DepTimestamp', 'timestamp'),
 ('ArrTimestamp', 'timestamp'),
 ('CRSArrTimestamp', 'timestamp')]

### {preprocess step} Cast bigint columns to int

In [14]:
int_col_list = ['ActualElapsedTime', 'AirTime', 'ArrDelay', 'CRSElapsedTime', 'DepDelay', 'Distance', 'LateAircraftAndCarrierDelay', 'NASAndWeatherDelay', 'SecurityDelay', 'TaxiIn', 'TaxiOut']
for col in int_col_list:
    data_trans_n_cast_cols = data_trans_n_cast_cols.withColumn(col, data_trans_n_cast_cols[col].cast('integer'))

### Analysis cells

In [157]:
data_trans_n_cast_cols.dtypes

[('ActualElapsedTime', 'int'),
 ('AirTime', 'int'),
 ('ArrDelay', 'int'),
 ('CRSElapsedTime', 'int'),
 ('DepDelay', 'int'),
 ('Dest', 'string'),
 ('Distance', 'int'),
 ('LateAircraftAndCarrierDelay', 'int'),
 ('NASAndWeatherDelay', 'int'),
 ('Origin', 'string'),
 ('SecurityDelay', 'int'),
 ('TailNum', 'string'),
 ('TaxiIn', 'int'),
 ('TaxiOut', 'int'),
 ('UniqueCarrier', 'string'),
 ('CRSDepTimeStamp', 'timestamp'),
 ('DepTimestamp', 'timestamp'),
 ('ArrTimestamp', 'timestamp'),
 ('CRSArrTimestamp', 'timestamp')]

In [158]:
data_trans_n_cast_cols.show(20)

+-----------------+-------+--------+--------------+--------+----+--------+---------------------------+------------------+------+-------------+-------+------+-------+-------------+-------------------+-------------------+-------------------+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|CRSElapsedTime|DepDelay|Dest|Distance|LateAircraftAndCarrierDelay|NASAndWeatherDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|    CRSDepTimeStamp|       DepTimestamp|       ArrTimestamp|    CRSArrTimestamp|
+-----------------+-------+--------+--------------+--------+----+--------+---------------------------+------------------+------+-------------+-------+------+-------+-------------+-------------------+-------------------+-------------------+-------------------+
|               53|     32|      -8|            65|       4| PIT|     205|                          0|                 0|   DCA|            0| N443US|     7|     14|           US|2002-10-10 15:45:00|2002-10-10 15:49:00|2

### Save dataframe to CSV again

In [ ]:
# will partition the dataframe into several csv files stored under a .csv directory
# to load in the preprocessed csv again you may use the path to the .csv directory
# spark will automatically load in all csv parts included in the directory
data_trans_n_cast_cols.write.csv('preprocessed_data.csv', header = True)

# started at 19:26

# Move preprocessing steps with spark to seperate notebook

In [32]:
del data

In [2]:
preproc_data = spark.read.csv('preprocessed_data.csv', inferSchema='true', header='true', mode='PERMISSIVE', encoding='ISO-8859-1').cache()

In [3]:
preproc_data.show(200)

+-----------------+-------+--------+--------------+--------+----+--------+---------------------------+------------------+------+-------------+--------+------+-------+-------------+-------------------+-------------------+-------------------+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|CRSElapsedTime|DepDelay|Dest|Distance|LateAircraftAndCarrierDelay|NASAndWeatherDelay|Origin|SecurityDelay| TailNum|TaxiIn|TaxiOut|UniqueCarrier|    CRSDepTimeStamp|       DepTimestamp|       ArrTimestamp|    CRSArrTimestamp|
+-----------------+-------+--------+--------------+--------+----+--------+---------------------------+------------------+------+-------------+--------+------+-------+-------------+-------------------+-------------------+-------------------+-------------------+
|              120|     86|       4|           115|      -1| OMA|     583|                          0|                 0|   DFW|            0|  N2BTAA|     4|     30|           AA|2002-04-22 16:06:00|2002-04-22 16:05:

In [4]:
preproc_data.count()

84210561